In [ ]:
import datajoint as dj

dj.config['external'] = dict(
              protocol='file',
              location = '/')
# external storage place

from pipeline import reso
from pipeline import experiment
from pipeline import mice
from pipeline import  notify, shared
import scanreader
from datajoint.jobs import key_hash
import matplotlib.pyplot as plt
import numpy as np
from pipeline.utils import galvo_corrections, signal, quality, mask_classification, performance
from pipeline.exceptions import PipelineException
import os
import time
import os
import multiprocessing as mp
from pipeline.exceptions import PipelineException
from pipeline.utils.signal import mirrconv
import tifffile
import cv2
dj.conn()

In [ ]:
patchtable=dj.schema("yizhou_patch",locals(),create_tables=True)

#right arm, about patch only

@patchtable
class PatchSession(dj.Manual): 
    definition = """ 
        -> mice.Mice    
        patch_session:int #session is different from reso session.
        ---
        # session here is one experiment, most likely, in one day.
        folder='':varchar(256) # the path to the folder that contains file of the cell.
        """
@patchtable
class Cell(dj.Manual):
    definition = """
    -> PatchSession #include animal id etc, patch session.
    ---
    notes='':varchar(256) # the notes about the cell. 
    # where does the dates etc go? notes?
    """
@patchtable
class Recording(dj.Manual): 
    definition = """
    -> Cell #include animal id etc, patch session, notes about the cell.
    ---
    filename='':varchar(256) 
    i1:external
    command:external
    vm:external
    frametimes:external
    ephystimes:external
    igain:int
    vgain:int
    ilowpass:int
    vlowpass:int
    vhighpass:int

    """
@patchtable
class Patchspikes(dj.Manual):
    definition = """ 
        -> Recording #which has animal, patch session, notes, raw file, setting, data.
        ---
        spike_ts:external # timestamps. on same clock as trace.
        """
    


    
    
#left arm, about imaging only

@patchtable
class Scaninfo(dj.Manual):
    definition = """   
    # those will be cp from reso.scaninfo, but only contain 1 entry
    animal_id:int
    session:int
    scan_idx:int
    pipe_version:int
    field:int #from correctionchannel.  field was a prim key
    ---
    #
    nfields                 : tinyint           # number of fields
    nchannels               : tinyint           # number of channels
    nframes                 : int               # number of recorded frames
    nframes_requested       : int               # number of requested frames (from header)
    px_height               : smallint          # lines per frame
    px_width                : smallint          # pixels per line
    um_height               : float             # height in microns
    um_width                : float             # width in microns
    x                       : float             # (um) center of scan in the motor coordinate system
    y                       : float             # (um) center of scan in the motor coordinate system
    fps                     : float             # (Hz) frames per second
    zoom                    : decimal(5,2)      # zoom factor
    bidirectional           : boolean           # true = bidirectional scanning
    usecs_per_line          : float             # microseconds per scan line
    fill_fraction           : float             # raster scan temporal fill fraction (see scanimage)
    valid_depth       : boolean           # whether depth has been manually check
    # above cp from reso.scaninfo
    channel:int  # from reso.correctionchannel, just one value so place it here.
    z:float # from scaninfo.field.
    delay_image:longblob #from scaninfo.field
    
    """    
        
@patchtable
class Segmentationtask(dj.Manual):
    definition = """ # defines the target of segmentation and the channel to use
    -> experiment.Scan
    -> shared.Field
    -> shared.Channel
    -> shared.SegmentationMethod
    ---
    -> experiment.Compartment
    """
    def fill(self, key, channel=1, segmentation_method=6, compartment='soma'):
        for field_key in (Field() & key).fetch(dj.key):
            tuple_ = {**field_key, 'channel': channel, 'compartment': compartment,
                      'segmentation_method': segmentation_method}
            self.insert1(tuple_, ignore_extra_fields=True, skip_duplicates=True)
    def estimate_num_components(self):
        """ Estimates the number of components per field using simple rules of thumb.

        For somatic scans, estimate number of neurons based on:
        (100x100x100)um^3 = 1e6 um^3 -> 100 neurons; (1x1x1)mm^3 = 1e9 um^3 -> 100K neurons

        For axonal/dendritic scans, just ten times our estimate of neurons.

        :returns: Number of components
        :rtype: int
        """
        # Get field dimensions (in microns)
        scan = (Scaninfo() & self & {'pipe_version': CURRENT_VERSION})
        field_height, field_width = scan.fetch1('um_height', 'um_width')
        field_thickness = 10  # assumption
        field_volume = field_width * field_height * field_thickness
        # Estimate number of components
        compartment = self.fetch1('compartment')
        if compartment == 'soma':
            num_components = field_volume * 0.0001
        elif compartment == 'axon':
            num_components = field_volume * 0.0005  # five times as many neurons
        elif compartment == 'bouton':
            num_components = field_volume * 0.001   # 10 times as many neurons
        else:
            PipelineException("Compartment type '{}' not recognized".format(compartment))
        return int(round(num_components))
    
        
@patchtable
class Rastercorrection(dj.Computed):
    definition = """ 
                             # animal_id, session, scan_idx, version
    -> Scaninfo
    # need correction channel, now the correction channel channel is in scaninfo as col, not an independent table.
    # now using the first channel to correct, normally its the green channel.
    ---
    raster_template     : longblob      # average frame from the middle of the movie
    raster_phase        : float         # difference between expected and recorded scan angle
    """    
    def make(self, key):
        from scipy.signal import tukey
        # Read the scan
        fps=(Scaninfo() & key).fetch1('fps')
        print("fps is ",fps)
        scan_filename = (experiment.Scan() & key).local_filenames_as_wildcard
        print(scan_filename)
        print(scanreader.core.expand_wildcard(scan_filename))
        scan = scanreader.read_scan(scan_filename, dtype=np.float32)
        # Select correction channel
        channel =  (Scaninfo()&key).fetch1('channel')-1
        print(key)
        field_id = key['field'] - 1
        # Load some frames from the middle of the scan
        middle_frame =  int(np.floor(scan.num_frames / 2))
        frames = slice(max(middle_frame - int(60*fps), 0), middle_frame + int(60*fps))
        startts=time.time()
        import tifffile
        scan_filename=scanreader.core.expand_wildcard(scan_filename)[0]
        # one recording session has one tiff
        f=tifffile.TiffFile(scan_filename)
        a=f.asarray(frames)
        print('pipe, a shape,', a.shape)
        mini_scan=np.moveaxis(a,0,2) #(f,x,y) to (y,x,f)
        print('pipe, mini_scan shape t1,', mini_scan.shape)
        mini_scan = mini_scan.astype(np.float32, copy=False)
#         mini_scan = scan[field_id, :, :, channel, frames]
        print('the time takes to get miniscan is:')
        print(time.time()-startts)
        # Create results tuple
        tuple_ = key.copy()
        # Create template (average frame tapered to avoid edge artifacts)
        taper = np.sqrt(np.outer(tukey(scan.image_height, 0.4),
                                 tukey(scan.image_width, 0.4)))
        anscombed = 2 * np.sqrt(mini_scan - mini_scan.min() + 3 / 8)  # anscombe transform
        template = np.mean(anscombed, axis=-1) * taper
        tuple_['raster_template'] = template
        # Compute raster correction parameters
        if scan.is_bidirectional:
            tuple_['raster_phase'] = galvo_corrections.compute_raster_phase(template,
                                                         scan.temporal_fill_fraction)
        else:
            tuple_['raster_phase'] = 0
        # Insert
        self.insert1(tuple_)
    def get_correct_raster(self):
        """ Returns a function to perform raster correction on the scan. """
        raster_phase = self.fetch1('raster_phase')
        fill_fraction = (Scaninfo() & self).fetch1('fill_fraction')
        if abs(raster_phase) < 1e-7:
            correct_raster = lambda scan: scan.astype(np.float32, copy=False)
        else:
            correct_raster = lambda scan: galvo_corrections.correct_raster(scan,
                                                             raster_phase, fill_fraction)
        return correct_raster

        
        
@patchtable
class  Motioncorrection(dj.Computed):
    definition = """ 
        ->Rastercorrection
        ---
        motion_template                 : longblob      # image used as alignment template
        y_shifts                        : longblob      # (pixels) y motion correction shifts
        x_shifts                        : longblob      # (pixels) x motion correction shifts
        y_std                           : float         # (pixels) standard deviation of y shifts
        x_std                           : float         # (pixels) standard deviation of x shifts
        outlier_frames                  : longblob      # mask with true for frames with outlier shifts (already corrected)
        align_time=CURRENT_TIMESTAMP    : timestamp     # automatic
        
    """
    
    def make(self, key):
        """Computes the motion shifts per frame needed to correct the scan."""
        from scipy import ndimage
        from pipeline.utils import galvo_corrections, signal, quality, mask_classification, performance
        import pyfftw
        from imreg_dft import utils
        import numpy as np
        from scipy import interpolate  as interp
        from scipy import signal
        from scipy import ndimage
        import tifffile
        # Read the scan
        scan_filename = (experiment.Scan() & key).local_filenames_as_wildcard
        scan = scanreader.read_scan(scan_filename)
        # Get some params
        px_height, px_width = (Scaninfo() & key).fetch1('px_height', 'px_width')
        channel = (Scaninfo() & key).fetch1('channel') - 1
        fps=(Scaninfo() & key).fetch1('fps')
        field_id = key['field'] - 1
        # Load some frames from middle of scan to compute template
        skip_rows = int(round(px_height * 0.10))  
        # we discard some rows/cols to avoid edge artifacts
        # so actually the motion correction is only looking at the center 80%
        # tested zero padding instead of taper, similar results
        # zero padding can preserve more data, use full mini scan 
        # taper is applied to mini scan, so less data than miniscan
        # i guess best way is to apply taper to full scan, than zero pad
        skip_cols = int(round(px_width * 0.10))
        middle_frame = int(np.floor(scan.num_frames / 2))
        fromframe=int(max(middle_frame - 100*fps, 0))
        toframe=int(middle_frame + 100*fps)
        print("gettign the frames in between ",fromframe,toframe)
        starttime=time.time()
        scan_filename=scanreader.core.expand_wildcard(scan_filename)[0]
        nchan=(Scaninfo()&key).fetch1('nchannels')
        correct_raster = (Rastercorrection() & key).get_correct_raster()
        # one recording session has one tiff
        f=tifffile.TiffFile(scan_filename)
        wholescan=f.asarray()# its the full scan
        wholescan=np.moveaxis(wholescan,0,2)#(f,x,y) to (y,x,f)
        # if there are multi channels, use the first channle (green)
        if nchan!=1:
            wholescan=wholescan[:,:,::nchan]
            # the channel frames are interlaved. green and red channel.
            # so ::nch is equal to 0::nch, taking all frames from first ch
        wholescan = correct_raster(wholescan)
        mini_scan=wholescan[:,:,fromframe:toframe] # now its mini scan temporally
        print('pipe, mini_scan shape ,', mini_scan.shape)
#         mini_scan=np.swapaxes(a,0,1) #(y,x,f) to (x,y,f)
#         print('pipe, mini_scan shape t2,', mini_scan.shape)
        mini_scan=mini_scan[skip_rows: -skip_rows, skip_cols: -skip_cols,:] # now its mini both temporally and spatially.
#         mini_scan = scan[field_id, skip_rows: -skip_rows, skip_cols: -skip_cols, channel,
#                          fromframe:toframe ]
        # right now the mini scan is a smaller frame, and only contain frames about several miniutes from the center.
        mini_scan = mini_scan.astype(np.float32, copy=False)
        # Create template
        mini_scan = 2 * np.sqrt(mini_scan - mini_scan.min() + 3 / 8)  # *
        template = np.mean(mini_scan, axis=-1)
        print('pipe, tem shape,', template.shape)
        template = ndimage.gaussian_filter(template, 0.7)  # **
        wholescan=wholescan[skip_rows: -skip_rows, skip_cols: -skip_cols,:]#prepare the whole scan into same shape as template.
        # * Anscombe tranform to normalize noise, increase contrast and decrease outliers' leverage
        # ** Small amount of gaussian smoothing to get rid of high frequency noise
        # now, had template
        # template is avg of frames from center of scan, both spatially and temporally, with o.7 gaussian smoothed.
      
        # with the template, start motion correction
        ################new parallel, but not improving speed, so not using it#####
        #####if consider using it, def a container class based on list, dont use the mp.queue and queue manager.########
        
#         print('entering mp')
#         def ycparallel_motion_shifts(chunks, results, raster_phase, fill_fraction, template):
#             """ Compute motion correction shifts to chunks of scan.
#             Function to run in each process. Consumes input from chunks and writes results to
#             results. Stops when stop signal is received in chunks.
#             :param queue chunks: Queue with inputs to consume.
#             :param list results: Where to put results.
#             :param float raster_phase: Raster phase used for raster correction.
#             :param float fill_fraction: Fill fraction used for raster correction.
#             :param np.array template: Template used to compute motion shifts.
#             :returns: (frames, y_shifts, x_shifts) tuples.
#             """
#             while True:
#                 # Read next chunk (process locks until something can be read)
#                 frames, chunk = chunks.get()
#                 if chunk is None:  # stop signal when all chunks have been processed
#                     return
#                 print('template shape ',template.shape)
#                 print(time.ctime(), 'Processing frames:', frames)
#                 # Correct raster
#                 chunk = chunk.astype(np.float32, copy=False)
#                 if abs(raster_phase) > 1e-7:
#                     chunk = galvo_corrections.correct_raster(chunk, raster_phase, fill_fraction)
#                 # Compute shifts
#                 y_shifts, x_shifts = yccompute_motion_shifts(chunk, template,
#                                                                              num_threads=2)
#                 # Add to results
#                 results.append((frames, y_shifts, x_shifts))
######## end of parallel function. to implent it, same ways as in performance.py###
######### start of single thread function.############################
        def yccompute_motion_shifts(data,template,method=1, in_place=True, num_threads=8):
            """ Compute shifts in y and x for rigid subpixel motion correction.
            :param np.array template: 2-d template image. Each frame in scan is aligned to this.
            for the mechanism, search for sub pixel shift/registration based on fft. it is a typical pixel registration method used in motion correction. for information rich image, gives promising results.
            """
            print(data.shape)
            # Get some paramsdata
            image_height, image_width, num_frames = data.shape
            taper = np.outer(signal.tukey(image_height, 0.2), signal.tukey(image_width, 0.2))
            taperlarge=(np.outer(signal.tukey(image_height, 0.6), signal.tukey(image_width, 0.6)))
            # Prepare fftw
            frame = pyfftw.empty_aligned((image_height, image_width), dtype='complex64')
            fft = pyfftw.builders.fft2(frame, threads=num_threads, overwrite_input=in_place,
                                       avoid_copy=True)
            ifft = pyfftw.builders.ifft2(frame, threads=num_threads, overwrite_input=in_place,
                                         avoid_copy=True)
            # Get fourier transform of template
            # print(template.shape)
            
            #the loop could be parallelized. its the rate limiting step now.
            # while the rate limiting step inside it is fft and ifft, which are already parallel.
            # check python mechanism of runing these
        # for i in range(111,112):
            # Compute correlation via cross power spectrum
#                 gaussian_k_size=
#                 gaussian_mu=0
#                 gaussian_sigma=
#                 image_freq = fft(
#                     np.mean(data[:, :, max(0,int(i-1)):min(num_frames,int(i+1))],2)
#                     *1/2
#                     +np.mean(data[:, :, max(0,int(i-0.01*fps)):min(num_frames,int(i+0.01*fps))],2)
#                     *1/3
#                     +np.mean(data[:, :, max(0,int(i-0.1*fps)):min(num_frames,int(i+0.1*fps))],2)
#                     *1/6
#                     * taper)
############ testing methods about windows functions.#######################
# for more information, search for window function and sliding window filtering####
# the idea is, for this high frequency sampling, single frames have very low snr
# think about like we need longger exposer to photo a night sky
# thus, using a window function to combine frames together to register is necessray, for sure.
# about different window functions. rectangle (simple avergaing) gives a slight enphasize on the center frame in fft. and other window functions could either focus more on the center, or more flat. (search for fft window function)
# ideally, we want to correct the center frame so we want focus more on the center
# about window size. larger window gives a stablier image, more signal. smaller winodw focus more on the center, but gives a crappy image.
# about taking mean or max. i dont have a clear understanding of how the data look at ( should have spent more time on data first ), but seems max gives higher snr in a smaller window. maybe because noise is possion with a smaller max, but signal has a much higher max ( when not firing)
# to do, window function/window size/ winodow mean max/preprosess the winodw
# preprocess, including smoothing, normalizing, transformations,
# mean idea, play with contrast, and try to make different window similar in value
# ( first test normalizing the whole data)
            if method==1: # using a pretty large rectangle window, with gaussian
                template_freq = fft(template * taper).conj() # we only need the conjugate
                abs_template_freq = abs(template_freq)
                eps = abs_template_freq.max() * 1e-15
                # Compute subpixel shifts per image
                y_shifts = np.empty(num_frames)
                x_shifts = np.empty(num_frames)
                starttime=time.time()
                for i in range(num_frames):
                    theimage=(
                    +np.max(data[:, :, max(0,int(i-0.05*fps)):min(num_frames,int(i+0.05*fps))],2)
                    *1
                    * taper)
                    image_freq=fft(ndimage.gaussian_filter(theimage,0.7))
                    cross_power = (image_freq * template_freq) / (abs(image_freq) * abs_template_freq + eps)
                    shifted_cross_power = np.fft.fftshift(abs(ifft(cross_power)))
                    # Get best shift
                    shifts = np.unravel_index(np.argmax(taperlarge*shifted_cross_power), shifted_cross_power.shape)
                    # here using a larger taper to focus on near shift 
                    # instead of moving large distance to match the avg frames of block with the template
                    #     print(shifts)
                    shifts = utils._interpolate(shifted_cross_power, shifts, rad=3)
                    # Map back to deviations from center
                    y_shifts[i] = shifts[0] - image_height // 2
                    x_shifts[i] = shifts[1] - image_width // 2
                    if i%10000==0:
                        print('now at ',i,time.time()-starttime)
                return y_shifts, x_shifts
                
            elif method==2: # using single frame
                template_freq = fft(template * taper).conj() # we only need the conjugate
                abs_template_freq = abs(template_freq)
                eps = abs_template_freq.max() * 1e-15
                # Compute subpixel shifts per image
                y_shifts = np.empty(num_frames)
                x_shifts = np.empty(num_frames)
                starttime=time.time()
                for i in range(num_frames):
                    theimage = (
    #                     np.mean(data[:, :, max(0,int(i-1)):min(num_frames,int(i+1))],2)
    #                     *1
    #                     +np.mean(data[:, :, max(0,int(i-0.01*fps)):min(num_frames,int(i+0.01*fps))],2)
    #                     *1/3
    #                     * taper
                        (data[:,:,i])
                    )
                    theimage=(theimage-np.min(theimage))/(np.max(theimage)-np.min(theimage))
                    image_freq=fft(ndimage.gaussian_filter(theimage,0.7))
    #                     view=np.zeros((image_height, image_width))
    #                     for m in range(image_height):
    #                         for n in range(image_width):
    #                                 view[m,n]=((m-image_height/2)**2+(n-image_width/2)**2)**0.5
    #                     view[image_height,image_width]=.9
                    cross_power = (image_freq * template_freq) / (abs(image_freq) * abs_template_freq + eps)
                    shifted_cross_power = np.fft.fftshift(abs(ifft(cross_power)))
                    # Get best shift
                    shifts = np.unravel_index(np.argmax(shifted_cross_power), shifted_cross_power.shape)
                    # here using a larger taper to focus on near shift 
                    # instead of moving large distance to match the avg frames of block with the template
                    #     print(shifts)
                    shifts = utils._interpolate(shifted_cross_power, shifts, rad=3)
                    # Map back to deviations from center
                    y_shifts[i] = shifts[0] - image_height // 2
                    x_shifts[i] = shifts[1] - image_width // 2
                    if i%10000==0:
                        print('now at ',i,time.time()-starttime)
                return y_shifts, x_shifts
        
            elif method==3: # using zero padding
                padwid=3
                frame = pyfftw.empty_aligned((image_height+2*padwid, image_width+2*padwid), dtype='complex64')
                fft = pyfftw.builders.fft2(frame, threads=num_threads, overwrite_input=in_place,
                                       avoid_copy=True)
                ifft = pyfftw.builders.ifft2(frame, threads=num_threads, overwrite_input=in_place,
                                         avoid_copy=True)
                template_freq = fft(np.pad((template),padwid,mode='constant')).conj() # we only need the conjugate
                abs_template_freq = abs(template_freq)
                eps = abs_template_freq.max() * 1e-15
                # Compute subpixel shifts per image
                y_shifts = np.empty(num_frames)
                x_shifts = np.empty(num_frames)
                starttime=time.time()
                for i in range(num_frames):
                    theimage=(
                    +np.max(data[:, :, max(0,int(i-0.025*fps)):min(num_frames,int(i+0.025*fps))],2)
                    *1)
                    image_freq=fft(ndimage.gaussian_filter(np.pad(theimage,padwid,mode='constant'),0.7))
                    cross_power = (image_freq * template_freq) / (abs(image_freq) * abs_template_freq + eps)
                    shifted_cross_power = np.fft.fftshift(abs(ifft(cross_power)))
                    shifts = np.unravel_index(np.argmax(shifted_cross_power), shifted_cross_power.shape)
                    shifts = utils._interpolate(shifted_cross_power, shifts, rad=3)
                    y_shifts[i] = shifts[0] - image_height // 2
                    x_shifts[i] = shifts[1] - image_width // 2
                    if i%10000==0:
                        print('now at ',i,time.time()-starttime)
                return y_shifts, x_shifts
              
# add tabs here. im on a old laptop and key registration is different                             y_shifts,x_shifts=yccompute_motion_shifts(thedata,template,method=1, in_place=True, num_threads=8)
        '''
########################### will only use one window after decide which one is best, implent it here #######################################
#                 image_freq = fft(
#                     +np.mean(data[:, :, max(0,int(i-0.01*fps)):min(num_frames,int(i+0.01*fps))],2)
#                     *1
#                     +np.max(data[:, :, max(0,int(i-0.05*fps)):min(num_frames,int(i+0.05*fps))],2)
#                     *1
#                     * taper)
                # using the moving rectangle/square wave block of frames
                # instead of by every frame.
                # using 0.025*fps, under the assumption of very less vibration frequence above 20hz
                # this is seen from fft of xy_shift.
                # estimation of these fast vib less than one pixel (most of the cases), so safely disregard them.
                
        ################define my function above to override the performance.motionshit#####
        # f = performance.parallel_motion_shifts  # function to map
        raster_phase = (Rastercorrection() & key).fetch1('raster_phase')
        fill_fraction = (Scaninfo() & key).fetch1('fill_fraction')
        kwargs = {'raster_phase': raster_phase, 'fill_fraction': fill_fraction,
                  'template': template}
#         os.system("taskset -p 0xff %d" % os.getpid())
############################
#         chunk_size_in_GB=0.05
#         num_processes = mp.cpu_count() - 1
#         print('Using', num_processes, 'processes')
#         # Calculate the number of frames per chunk
        y=slice(skip_rows, -skip_rows)
        x=slice(skip_cols, -skip_cols)
#         one_frame = scan[field_id, y, x, channel, 0]
#         # just one frame so keep using scanreader is fine
#         bytes_per_frame = np.prod(one_frame.shape) * 4 # 4 bytes per pixel
#         chunk_size = int(round((chunk_size_in_GB * 1024**3) / bytes_per_frame))
        num_frames = scan.num_frames
        print('num_frames ',num_frames)
        thedata=wholescan[skip_rows: -skip_rows, skip_cols: -skip_cols,:]
        print('thedata shape3',thedata.shape)
#         chunks=[]
#         for i in range(0, chunk_size, chunk_size):
#             frames = slice(i, min(i + chunk_size, chunk_size))
#             thedata=scanfile.asarray(frames)
#             thedata=np.swapaxes(thedata,0,2)
#             thedata=thedata[skip_rows: -skip_rows, skip_cols: -skip_cols,:]
#             chunks.append((thedata))
        #############################
#         # Create a Queue to put in new chunks and a list for results
#         scan_filename=scanreader.core.expand_wildcard(scan_filename)[0]
#         scanfile=tifffile.TiffFile(scan_filename)
#         # mp init
#         manager = mp.Manager()
#         chunks = manager.Queue(maxsize=20)
#         results = manager.list()
#         pool = []
#         for i in range(num_processes):
# # def the function to call to parallel motion correction
# # the args are not ready yet. and function not called yet
# #             p = mp.Process(target=performance.parallel_motion_shifts, args=(chunks, results),kwargs=kwargs)
#             p = mp.Process(target=ycparallel_motion_shifts, args=(chunks, results),kwargs=kwargs)
#             p.start()
#             print(p.pid)
#             pool.append(p)
#         # Produce data
#         num_frames = scan.num_frames
#         for i in range(0, num_frames, chunk_size):
#             frames = slice(i, min(i + chunk_size, num_frames))
#             thedata=scanfile.asarray(frames)
#             thedata=np.swapaxes(thedata,0,2)
#             thedata=thedata[skip_rows: -skip_rows, skip_cols: -skip_cols,:]
#             chunks.put((frames,thedata))
#         # Queue STOP signal
#         for i in range(num_processes):
#             chunks.put((None, None))
#         # Wait for processes to finish
#         for p in pool:
#             p.join()

#####################
#         import tifffile
#         scan_filename=scanreader.core.expand_wildcard(scan_filename)[0]
#         f=tifffile.TiffFile(scan_filename)
#         manager = mp.Manager()
#         chunks = manager.Queue(maxsize=20)
#         num_frames=200000
#         chunk_size=10000
#         for i in range(0, num_frames, chunk_size):
#             frames = slice(i, min(i + chunk_size, num_frames))
#             chunks.put(f.asarray(frames))
#############################      
#         results = performance.map_frames(f, scan, field_id=field_id,
#                                          y=slice(skip_rows, -skip_rows),
#                                          x=slice(skip_cols, -skip_cols), channel=channel,
#                                          kwargs=kwargs,chunk_size_in_GB=0.005)
        # Reduce
#         y_shifts = np.zeros(scan.num_frames)
#         x_shifts = np.zeros(scan.num_frames)
#         for frames, chunk_y_shifts, chunk_x_shifts in results:
#             y_shifts[frames] = chunk_y_shifts
#             x_shifts[frames] = chunk_x_shifts
'''
        def postshift(tuple_,key):
            if type(tuple_)==dict:
                y_shifts=tuple_['y_shifts']
                x_shifts=tuple_['x_shifts']
            elif type(tuple_)==tuple:
                (y_shifts,x_shifts)=tuple_
            # Detect outliers
            max_y_shift, max_x_shift = 20 / (reso.ScanInfo() & key).microns_per_pixel
            y_shifts, x_shifts, outliers = galvo_corrections.fix_outliers(y_shifts, x_shifts,max_y_shift,max_x_shift)
            # Center shifts around zero
            y_shifts -= np.median(y_shifts)
            x_shifts -= np.median(x_shifts)
            # Create results tuple
            tuple_ = key.copy()
            tuple_['field'] = field_id + 1
            tuple_['motion_template'] = template
            tuple_['y_shifts'] = y_shifts
            tuple_['x_shifts'] = x_shifts
            tuple_['outlier_frames'] = outliers
            tuple_['y_std'] = np.std(y_shifts)
            tuple_['x_std'] = np.std(x_shifts)
            return tuple_
        def mcagain(wholescan,tuple_,fromframe,toframe,method):
            wholescan=galvo_corrections.correct_motion(wholescan,tuple_['x_shifts'], tuple_['y_shifts'])
            mini_scan=wholescan[:,:,fromframe:toframe] # mini spatially to mini spatially/temporaly
            mini_scan = mini_scan.astype(np.float32, copy=False)
            mini_scan = 2 * np.sqrt(mini_scan - mini_scan.min() + 3 / 8)
            template = np.mean(mini_scan, axis=-1)
            template = ndimage.gaussian_filter(template, 0.7)
            # define the method used in second motion correction here.
            y_shifts2, x_shifts2=yccompute_motion_shifts(wholescan,template,method=method, in_place=True, num_threads=8)
            tuple_['x_shifts']=tuple_['x_shifts']+x_shifts2
            tuple_['y_shifts']=tuple_['y_shifts']+y_shifts2
            return tuple_
        # 1st round of motion correction
        y_shifts, x_shifts=yccompute_motion_shifts(wholescan,template,method=1, in_place=True, num_threads=8)
        tuple_=postshift((y_shifts, x_shifts),key)
        tuple_=mcagain(wholescan,tuple_,fromframe,toframe,method=2)
        tuple_=postshift(tuple_,key)
        ####################################################
        # # if use second time motion correction ###################   
        # idea is, a typical motion correction way is to use recurrsive till the shifts are below a certain threshold ( mag, std, etc) from articles iv read, about 6-8 times will give a nearly perfect shifts.
        # so, im guess running it at least 2 times i would see an improvement. but not significant
        # plan is, either using a same, good, method to run multi times, since the method is good, it works better with a sharper template, gives better result
        # or, using a high frequncy noise prone method first, get rid of low freq motion to achieve a slighly better template, then a smaller window high temporal resolution method to further correct the rest high freq motion.
        # wholescan is the spatially smaller version of full scan, already rastercorrected
        # now, correct it with last result.
       
#         ############################## if more desired.#######
#         mc_count=0
#         while mc_count>0:
#             mc_count-=1
#             wholescan=galvo_corrections.correct_motion(wholescan,x_shifts2, y_shifts2)
#             mini_scan=wholescan[:,:,fromframe:toframe] # mini spatially to mini spatially/temporaly
#             mini_scan = mini_scan.astype(np.float32, copy=False)
#             mini_scan = 2 * np.sqrt(mini_scan - mini_scan.min() + 3 / 8)
#             template = np.mean(mini_scan, axis=-1)
#             print(mc_count,'mc_count, tem shape,', template.shape)
#             template = ndimage.gaussian_filter(template, 0.7)
#             # define the method used in second motion correction here.
#             y_shifts2, x_shifts2=yccompute_motion_shifts(wholescan,template,method=1, in_place=True, num_threads=8)
#             print(y_shifts.shape,'shift done. shape')
#             x_shifts=x_shifts+x_shifts2
#             y_shifts=y_shifts+y_shifts2

        # about outliers and shifts
        # ideally, the mean of shifts is 0, small amplitude, reasonable frequency
        # because vibration generally around the center, avg to 0
        # small vibration insteard of large. based on um. around 5 um?
        # should be less than 20 hz frequncy?
        # tested using a butter low pass filter to apply to shifts, not very good.
        # tested fft, 2nd threshold cut off, ifft, even worse.( because of ft edge artifact.)
        # maybe because the shift is not good, and filter wont rescue it.
        # things to test, find the frequence from ca scan shifts, (better filter), 
        # Insert
        self.insert1(tuple_)
        # Notify after all fields have been processed
        scan_key = {'animal_id': key['animal_id'], 'session': key['session'],
                    'scan_idx': key['scan_idx'], 'pipe_version': key['pipe_version']}
        if len(Motioncorrection - Scaninfo & scan_key) > 0:
            self.notify(scan_key, scan.num_frames, scan.num_fields)
            
    @notify.ignore_exceptions
    def notify(self, key, num_frames, num_fields):
        fps = (Scaninfo() & key).fetch1('fps')
        seconds = np.arange(num_frames) / fps
        fig, axes = plt.subplots(num_fields, 1, figsize=(15, 4 * num_fields), sharey=True)
        axes = [axes] if num_fields == 1 else axes # make list if single axis object
        for i in range(num_fields):
            y_shifts, x_shifts = (self & key & {'field': i + 1}).fetch1('y_shifts',
                                                                        'x_shifts')
            axes[i].set_title('Shifts for field {}'.format(i + 1))
            axes[i].plot(seconds, y_shifts, label='y shifts')
            axes[i].plot(seconds, x_shifts, label='x shifts')
            axes[i].set_ylabel('Pixels')
            axes[i].set_xlabel('Seconds')
            axes[i].legend()
        fig.tight_layout()
        img_filename = '/tmp/' + key_hash(key) + '.png'
        fig.savefig(img_filename, bbox_inches='tight')
        plt.close(fig)
        msg = 'motion shifts for {animal_id}-{session}-{scan_idx}'.format(**key)
        slack_user = notify.SlackUser() & (experiment.Session() & key)
        slack_user.notify(file=img_filename, file_title=msg)
   
    def get_correct_motionf(self):
        """ Returns a function to perform motion correction on scans. """
        import scipy
        x_shifts, y_shifts = self.fetch1('x_shifts', 'y_shifts')
        b, a = scipy.signal.butter(4, 20, 'low',fs=400)
        x_shifts=scipy.signal.lfilter(b,a,x_shifts)
        y_shifts=scipy.signal.lfilter(b,a,y_shifts)
        return lambda scan, indices=slice(None): galvo_corrections.correct_motion(scan,
                                                 x_shifts[indices], y_shifts[indices])
    def get_correct_motion(self):
        """ Returns a function to perform motion correction on scans. """
        x_shifts, y_shifts = self.fetch1('x_shifts', 'y_shifts')
        return lambda scan, indices=slice(None): galvo_corrections.correct_motion(scan,
                                                 x_shifts[indices], y_shifts[indices])

    def getvideo(self,fname,filter=False):
        correct_raster = (Rastercorrection()&self).get_correct_raster()
        if filter:
            correct_motion = self.get_correct_motionf()
        else:
            correct_motion = self.get_correct_motion()
        nchan=(Scaninfo()&self).fetch1('nchannels')
        fps=(Scaninfo()&self).fetch1('fps')
        dim=((Scaninfo()&self).fetch1('px_width'),(Scaninfo()&self).fetch1('px_height'))
        f=tifffile.TiffFile(scanreader.core.expand_wildcard((experiment.Scan() & self ).local_filenames_as_wildcard)[0])
        wholecan=f.asarray()
        wholecan=np.moveaxis(wholecan,0,2)
        wholecan=wholecan[:,:,::nchan]
        num_frames=(Scaninfo()&self).fetch1('nframes')
        savevideo(fps,dim,num_frames,correct_motion(correct_raster(wholecan)),
                  video_filename=fname)
    def savevideo(fps,dim,num_frames,clipdata,video_filename='original.avi'):
        fourcc=cv2.VideoWriter_fourcc('M',"J","P","G")
        out=cv2.VideoWriter(video_filename,fourcc,fps,dim)
        for i in range(num_frames):
            ardata=cv2.normalize(clipdata[:, :, i],None,255,0,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_8U)
            ardata3c=cv2.merge([ardata,ardata,ardata])
            out.write(ardata3c)
        out.release()

@patchtable
class Segmentation(dj.Manual):
    definition = """ 
        ->Motioncorrection
        # segmentation task
        ---
        time:timestamp
        """
#     class Mask(dj.Part):
#         definition = """ 
#         -> Segmentation
#         mask_id         : smallint
#         ---
#         pixels          : longblob      # indices into the image in column major (Fortran) order
#         weights         : longblob      # weights of the mask at the indices above
#         """
#     class Method1(dj.Part):
#         definition = """ 
#         ->Segmentation
#         ---
#         paramenter:int
#         """
@patchtable
class Trace(dj.Manual):
    definition = """ 
        ->Segmentation
        ---
        trace:external
    """  
@patchtable
class Tracespikes(dj.Manual):
    definition = """ 
        ->Trace
        ---
        spikets:external
    """
# combining primary keys
@patchtable
class Ephys2p(dj.Manual):
    definition = """ 
        ->Recording
        ->Segmentation
        ---
        
    """ 
    @property
    def key_source(self):
        return Recording()# & {'pipe_version': CURRENT_VERSION}
        # only using the key from xxx() to find the key in other tb
    
# dj.ERD(patchtable)

In [ ]:
# %time Rastercorrection.populate()
# %time Motioncorrection.populate()


In [ ]:

# scaninfokeys=(reso.ScanInfo() & {'animal_id':20053,'session':3,'scan_idx':1}).fetch1()


correctionchkeys=(reso.CorrectionChannel() & {'animal_id':20053,'session':3,'scan_idx':1}).fetch1()
scaninfokeys['channel']=correctionchkeys['channel']
scaninfokeys['field']=correctionchkeys['field']
fieldkeys=(reso.ScanInfo.Field() & {'animal_id':20053,'session':3,'scan_idx':1}).fetch1()
scaninfokeys['z']=fieldkeys['z']
scaninfokeys['delay_image']=fieldkeys['delay_image']


Scaninfo.insert1(scaninfokeys,skip_duplicates=True)

Rastercorrection.populate()
Rastercorrection().fetch1()
Scaninfo().fetch1('fps')

In [ ]:
plt.plot(y[1000:1100])

In [ ]:
compkey={'animal_id':18252,'session':2,'scan_idx':27}

cy=(reso.MotionCorrection()&compkey).fetch1('y_shifts')
cfps=(reso.ScanInfo()&compkey).fetch1('fps')
secax=list(range(len(cy)))/cfps
plt.plot(secax[1000:1100],cy[1000:1100])
cfps

In [ ]:
def motiontraceplot(fromto):
    x=Motioncorrection().fetch1('x_shifts')
    fps=Scaninfo().fetch1('fps')
    secunitaxis=list(range(len(x)))/fps
    plt.plot(secunitaxis[fromto[0]:fromto[1]],x[fromto[0]:fromto[1]])
    

In [ ]:
motiontraceplot([0,1200])

In [ ]:
plt.imshow(chunk[:, :, i])
plt.imshow(np.fft.fftshift(abs(ifft(fft(chunk[:, :, i])*template_freq))))

In [ ]:
plt.imshow(np.mean(chunk[:, :, 100:int(100+.05*fps)],2))
chunk[:, :, 100:int(100+fps)].shape

In [ ]:
np.argmax(shifted_cross_power), shifted_cross_power.shape

In [ ]:
np.unravel_index(np.argmax(shifted_cross_power), shifted_cross_power.shape)

In [ ]:
import math
plt.imshow(abs(np.log(1+fft(chunk[:, :, i]))))

In [ ]:
scanreader.core.expand_wildcard((experiment.Scan() & (reso.ScanInfo & 'fps>20').fetch('KEY')[3]).local_filenames_as_wildcard)[0]

In [ ]:
key={'animal_id':20053,'session':3,'scan_idx':1}
correct_raster = (Rastercorrection()).get_correct_raster()
correct_motion = (Motioncorrection() ).get_correct_motion()
f=tifffile.TiffFile(scanreader.core.expand_wildcard((experiment.Scan() & key).local_filenames_as_wildcard)[0])
wholecan=f.asarray()
wholecan=np.swapaxes(wholecan,0,2)


In [ ]:
plt.imshow(np.mean(wholecan[:,:,10000:13000],2))

In [ ]:
rcrez=np.mean((correct_raster(wholecan)),2,)

In [ ]:
plt.imshow(rcrez,vmin=40)

In [ ]:
mcrez=(correct_motion(correct_raster(wholecan)))

In [ ]:
plt.imshow(mcrez*mcrez)

In [ ]:
plt.imshow((mcrez-rcrez)[2:-2,2:-2])

In [ ]:
subrez=np.subtract(   (correct_motion(correct_raster(wholecan))), (correct_raster(wholecan))  )

In [ ]:
plt.imshow(np.mean(subrez[1:-1,1:-1,8000:10000],2))

In [ ]:
plt.imshow(np.mean(mcrez,2), vmin=40)

In [ ]:
m,n=np.where((np.mean(mcrez,2))>90)


zeroarry=np.zeros(mcrez.shape)
for mm,nn in zip(m,n):
    zeroarry[mm,nn,:]=mcrez[mm,nn,:]
             #now zeroarry is like a neuron
        # can extract trace
# plt.imshow(zeroarry[:,:,3])

trace=[]
for frames in range(mcrez.shape[-1]):
    trace.append(np.mean(zeroarry[:,:,frames]))
    
plt.imshow(np.mean(zeroarry,2))

In [ ]:
fig=plt.figure(figsize=(50,20))
plt.plot(trace)

In [ ]:
import h5
trace=np.asarray(trace)
data=h5.readephys('oneCell10_0.h5')
patchcell={}
patchcell['depth']=190
patchcell['power']=60
vt = h5.ts2sec(data['data']['ts'], is_packeted=True)
vm=data['data']['v1']
peaks=(np.diff(data['data']['scanImage']))
peaks2=np.argwhere((np.diff(data['data']['scanImage'])) > 2.495)
ft=vt[peaks2]
patchcell['frametimes']=ft
patchcell['ephystimes']=vt
patchcell['vm']=vm*1000
patchcell['trace']=trace
fig=plt.figure(figsize=(50,20))
plt.plot(patchcell['ephystimes'],patchcell['vm']*10,alpha=0.8)
plt.plot(patchcell['frametimes'],patchcell['trace'],alpha=0.8)
plt.scatter(patchcell['ephystimes'][firstspkindex],patchcell['vm'][firstspkindex]*10,color = 'r',s=10)

In [ ]:
patchcell['trace']=(trace-np.mean(trace))*(trace-np.mean(trace))-np.mean(patchcell['vm']*10)
fig=plt.figure(figsize=(50,20))
plt.plot(patchcell['ephystimes'],patchcell['vm']*10,alpha=0.8)
plt.plot(patchcell['frametimes'],patchcell['trace'],alpha=0.8)
plt.scatter(patchcell['ephystimes'][firstspkindex],patchcell['vm'][firstspkindex]*10,color = 'r',s=10)



In [ ]:
from scipy.signal import butter, lfilter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

rez=butter_bandpass_filter(trace,5.0,30,396)

In [ ]:
fig=plt.figure(figsize=(50,20))
plt.plot(patchcell['frametimes'],patchcell['trace'],alpha=0.8)
plt.plot(patchcell['frametimes'],rez,alpha=0.8)
# for i in patchcell['ephystimes'][firstspkindex]:
#         plt.axvline(x=i,ymin=0,ymax=10,color = 'r',alpha=0.3)

In [ ]:
fig=plt.figure(figsize=(50,20))
plt.plot(patchcell['ephystimes'],patchcell['vm']*10,alpha=0.8)
# plt.plot(patchcell['frametimes'],patchcell['trace'],alpha=0.8)
plt.scatter(patchcell['ephystimes'][firstspkindex],patchcell['vm'][firstspkindex]*10,color = 'r',s=10)
plt.plot(patchcell['frametimes'],rez*15+40,alpha=0.8)

In [ ]:
fig=plt.figure(figsize=(50,20))
plt.plot(patchcell['ephystimes'][4000000:4030000],patchcell['vm'][4000000:4030000]*10,alpha=0.8)
# plt.plot(patchcell['frametimes'],patchcell['trace'],alpha=0.8)
plt.scatter(patchcell['ephystimes'][firstspkindex][4000000:4030000],patchcell['vm'][firstspkindex][4000000:4030000]*10,color = 'r',s=20)
plt.plot(patchcell['frametimes'][130700:132000],rez[130700:132000]*15+20,alpha=0.8)

In [ ]:
plt.plot(patchcell['frametimes'][130000:130100],alpha=0.8)
plt.plot(list(range(patchcell['frametimes'][130000],patchcell['frametimes'][130100]),alpha=0.8))

In [ ]:
patchcell['frametimes'][130100]-patchcell['frametimes'][130000]
data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAYoAAAD8CAYAAABpcuN4AAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDIuMS4xLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvAOZPmwAAH3xJREFUeJzt3X2w3NV93/H3R08YsAh2ENQGYUQiUYsSKeiicUogQIMabBc1ISR4HOJEAcUeJZbVQmsyQJPMeJo6uLHctJYVWWoyU7BJQC5uA0KmMSQjP/QqIFvIcnmwXISIJeEKnAfr3t399I/fWWsRe+9d6V5p9+79vGY89+zZs2d/Zxafr87D73dkm4iIiJFM6/YFREREb0ugiIiIUSVQRETEqBIoIiJiVAkUERExqgSKiIgYVQJFRESMKoEiIiJGlUARERGjmtHtC5gIZ511li+44IJuX0ZExKSyffv2g7bnjFWuLwLFBRdcwODgYLcvIyJiUpH07U7KZeopIiJGlUARERGjSqCIiIhRJVBERMSoEigiImJUCRQRETGqBIqIiBhVX9xHERHRC/a/+n2+8I39NE7iEdNv++HTuGL+mPfMjUsCRUTEBNny9N9w/+BepJP3nT/5o2f1RqCQtAf4HlAHarYHJC0C1gFvBPYA77X9qqRZwKeAAaABrLb9xTZ1tv18ee8O4NfK933Q9pbjb2JExMlxuNbg1JnTuf/9P9HtS5lQx7JGcbXtxbYHyusNwIdtXwJsBm4v+bcClPxrgY9Javc9bT8vaSFwE3Ax8DPAf5E0/diaFRFx8g3VG8yccRKHEyfJeBazLwKeKOmtwA0lvRB4DMD2fuAQ1eii088vBz5j+7DtbwHPAkvHcZ0RESfFcM3MnN5/e4Q6bZGBRyVtl7Sy5O0Eri/pG4G5Jb0DWC5phqR5wJKW91qN9PlzgRdayu0teRERPW243mDWFA4Ul9u+FLgOWCXpSmBFSW8HZgNDpexGqs59EPg4sA2otalzpM+3G7e9bguBpJWSBiUNHjhwoMNmREScOMP1BjNn9F+g6Ggx2/a+8ne/pM3AUtv3AMsAJC0A3lXK1IA1zc9K2gY806bO3e0+TxVkWkcg5wH72nx+PbAeYGBg4OTtRYuIGMHhWoNTpuKIQtLpkmY301Sd+05JZ5e8acCdVDuYkHRaKYeka6l2Se1qU2/bzwMPATdJOqVMXc0HvjquVkZEnATD9caUXaM4B/grSTuoOuz/afsR4D2S/g+wm+pf/JtK+bOBv5b0DeDfAjc3K5K0QVJzYbvt520/DdwP7AIeAVbZro+vmRERJ95QrT93Pckn8Q7CE2VgYMA54S4iuu1Dn3mSN59+Cnf/i4XdvpSOSNrecsvDiPpvjBQR0SXDdffliCKBIiJigkzZxeyIiOjMVF7MjoiIDlSL2f3XrfZfiyIiuiQjioiIGFX1CI8sZkdERBv1hmkYZmXqKSIi2hmqNQAy9RQREe0N1atAkRFFRES0NVzPiCIiIkbRnHqayudRRETEKIYz9RQREaPJYnZERIwqi9kRETGq4Xp1ZMPM3HAXERHtTPnFbEl7JH1d0lOSBkveIklfKvmfl3RGyZ8laVPJ3yHpqhHqXCzpy806JS0t+VdJeqXkPyXp7glqa0TECdPP22NnHEPZq20fbHm9AbjN9uOSVgC3A3cBtwLYvqSci/2wpMtsN46q76PA79h+WNI7y+urynt/afvdx9GeiIiu+MFidtYoXuMi4ImS3grcUNILgccAbO8HDgHtjtozcEZJ/xDVudkREZPSDxaz+3BE0WmLDDwqabuklSVvJ3B9Sd8IzC3pHcBySTMkzQOWtLzX6kPA70t6AbgHuKPlvZ8o01YPS7q43QVJWlmmrAYPHDjQYTMiIk6M4QQKLrd9KXAdsErSlcCKkt4OzAaGStmNwF5gEPg4sA2otanzA8Aa23OBNcCnS/5fA2+zvQj4T8Dn2l2Q7fW2B2wPzJkzp8NmREScGFP+hjvb+8rf/cBmYKnt3baX2V4C3Ac8V8rUbK+xvdj2cuBM4Jk21b4PeLCk/xRYWj7/qu2/Lek/B2ZKOuu4WxgRcRIM16bw9lhJp0ua3UwDy4CdZaEaSdOAO4F15fVppRySrgVqtne1qXof8FMlfQ0lmEj6R5JU0kvLNb583C2MiDgJDtcbTBNMn9Z/gaKTXU/nAJtL3z0DuNf2I5JWS1pVyjwIbCrps4EtkhrAi8DNzYokbQDW2R6k2h21VtIM4PtAc+3j54EPSKoB/wDcZNvjaWRExIk2XKuOQS19ZV8ZM1DYfh5Y1CZ/LbC2Tf4eqh1R7eq6pSX9V1QL3UeX+UPgD8e6roiIXjLUp+dlQ+7MjoiYEMO1Rl8uZEMCRUTEhBjOiCIiIkZzuN7glIwoIiJiJMM19+XWWEigiIiYEEP1eqaeIiJiZMM19+UDASGBIiJiQgzXG335nCdIoIiImBBD9WyPjYiIUVTbY7OYHRERIxiqNZg1fXq3L+OESKCIiJgAw3Uzc0ZGFBERMYJqRNGfXWp/tioi4iTLYnZERIyo0TD1hnPDXUREtDfUx+dlQwJFRMS4Nc/LntJ3ZkvaI+nrkp6SNFjyFkn6Usn/vKQzSv4sSZtK/g5JV41Q52JJX27WWY49RZVPSHpW0tckXTpBbY2IOCGGas0RRXY9XW17se2B8noD8GHblwCbgdtL/q0AJf9a4GPlXO2jfRT4HduLgbvLa4DrgPnlfyuBTx7DNUZEnHTD9eq05qxRvN5FwBMlvRW4oaQXAo8B2N4PHAIGXvdpMHBGSf8QsK+klwN/4sqXgTMlvWUc1xkRcUI1RxRTPVAYeFTSdkkrS95O4PqSvhGYW9I7gOWSZkiaR3Uu9lxe70PA70t6AbgHuKPknwu80FJub8l7DUkry5TV4IEDBzpsRkTExBtulKmnqbxGAVxu+1KqaaFVkq4EVpT0dmA2MFTKbqTq3AeBjwPbgFqbOj8ArLE9F1gDfLrkt5vk8+sy7PW2B2wPzJkzp8NmRERMvH4fUczopJDtfeXvfkmbgaW27wGWAUhaALyrlKlRdfyU97YBz7Sp9n3A6pL+U6o1D6iCTOsI5DyOTEtFRPSc5q6nKXsUqqTTJc1upqmCw05JZ5e8acCdwLry+rRSDknXAjXbu9pUvQ/4qZK+hiPB5CHgl8vup3cAr9h+6XgbGBFxov1ge+wUHlGcA2yW1Cx/r+1HJK2WtKqUeRDYVNJnA1skNYAXgZubFUnaAKyzPUi1O2qtpBnA96l2OAH8OfBO4Fng74FfHUf7IiJOuMM/mHrqz+2xYwYK288Di9rkrwXWtsnfQ7Ujql1dt7Sk/4pqofvoMgZWHZ0fEdGrmttjp/pidkREjODIDXf92aX2Z6siIk6i5hpFRhQREdFWvy9m92erIiJOosN9fh9Ff7YqIuIkOjKi6M9dTwkUERHjNFRrMHO6KLcR9J0EioiIcarV+/d0O0igiIgYt34+LxsSKCIixm2o1ujbeygggSIiYtyG641MPUVExMiGapl6ioiIUWREERERo6oWs/tzaywkUEREjNtQzVnMjoiIkfX71FNHR6FK2gN8D6hTnVg3IGkR1al2bwT2AO+1/aqkWcCngAGgAay2/cU2dX6WI+dWnAkcsr1Y0gXAN4Bvlve+bPv9x9O4iIiTod8XszsKFMXVtg+2vN4A3Gb7cUkrgNuBu6hOrsP2JeW41IclXWa70VqZ7V9spiV9DHil5e3nbC8+xrZERHTFcL3BjD4eUYynZRcBT5T0VuCGkl4IPAZgez9wiGp00Zaqh6P8AnDfOK4lIqJrhuoNTunjEUWnLTPwqKTtkppnW+8Eri/pG4G5Jb0DWC5phqR5VMedzmVkVwDfsf1MS948SU9KelzSFR1eY0REVzQfCtivOp16utz2vjKVtFXSbmAF8AlJdwMPAUOl7Ebg7cAg8G1gG1Abpe738NrRxEvA+bZflrQE+Jyki22/2vqhErBWApx//vkdNiMiYuL1+2J2Ry2zva/83Q9sBpba3m17me0lVB39c6VMzfYa24ttL6daqH6mXb2SZgA/B3y25bsO2365pLeXehe0uab1tgdsD8yZM6fzFkdETCDbDNfd14vZY7ZM0umSZjfTwDJgZxldIGkacCfVDigknVbKIelaql1Su0ao/qeB3bb3tnzfHEnTS/pCYD7w/HG2LyLihBquG+jf0+2gs6mnc4DN5UCOGcC9th+RtFrSqlLmQWBTSZ8NbJHUAF4Ebm5WJGkDsM72YMm6idcvYl8J/K6kGtV23Pfb/u6xNy0i4sRrnm7Xz4vZYwYK288Di9rkrwXWtsnfw5H7I45+75ajXv9KmzIPAA+MdV0REb3gyDGo/Rso+rdlEREnwVAtgSIiIkYxVEYUU3oxOyIiRnZkRNG/91EkUEREjENz11M/L2b3b8siIk6CLGZHRMSoDmcxOyIiRpMRRUREjKoZKHLCXUREtDWc7bERETGabI+NiIhRDZXtsRlRREREW8PZ9RQREaMZymJ2RESMZrjeYPo0MW1a/65RdHoUapxAttn3yvepl7nOiJg8Dv7tUF+PJiCBoid86bmX+fcP7+72ZUTEcfrhN87q9iWcUB0FCkl7gO9RnThXsz0gaRHV8advBPYA77X9qqRZwKeAAaABrLb9xTZ1fpYjBxydCRyyvbi8dwfwa+X7Pmh7y/E2cDL47t8PAfCb1/wop81K7I6YbM5706ndvoQT6lh6pattH2x5vQG4zfbjklYAtwN3AbcC2L6knKv9sKTLbDdaK7P9i820pI8Br5T0QqojUi8G3gp8QdIC2/Vjb97k0Lxh54r5czh11vQuX01ExGuNZ2LtIuCJkt4K3FDSC4HHAGzvBw5RjS7aUnUY9y9w5Ozs5cBnbB+2/S3gWWDpOK6z5w3Xmoez9+9iWERMXp0GCgOPStouaWXJ2wlcX9I3AnNLegewXNIMSfOAJS3vtXMF8B3bz5TX5wIvtLy/t+T1rcP1BtME0/t410RETF6dTj1dbntfmUraKmk3sAL4hKS7gYeAoVJ2I/B2YBD4NrANqI1S93s4MpoAaNdbvm47UAlYKwHOP//8DpvRm4ZrDWZOn0Y1uIqI6C0dBQrb+8rf/ZI2A0tt3wMsA5C0AHhXKVMD1jQ/K2kb8MzrKq3emwH8HNWoo2kvrx2BnAfsa3NN64H1AAMDA5N6X+lQvdHXd3VGxOQ2Zu8k6XRJs5tpquCws4wukDQNuJNqBxSSTivlkHQt1S6pXSNU/9PAbtt7W/IeAm6SdEqZupoPfPW4WjdJDNcaff2cmIiY3DoZUZwDbC7TIjOAe20/Imm1pFWlzIPAppI+G9giqQG8CNzcrEjSBmCd7cGSdROvnXbC9tOS7gd2UU1ZrernHU9Q7XrKiCIietWYgcL288CiNvlrgbVt8vdw5P6Io9+75ajXvzJCuY8AHxnr2vrF4Xqjrw9mj4jJLb1TDxiuOVtjI6JnJVD0gKF6PVNPEdGz0jv1gOGas5gdET0rvVMPyGJ2RPSy9E49IIvZEdHL0jv1gOad2RERvSi9Uw/IndkR0cvSO/WA4XruzI6I3pXeqQfkPoqI6GUJFD0gi9kR0cvSO3VZvWEaDWeNIiJ6VnqnLmseg5pAERG9Kr1Tlw2VQJHF7IjoVemdumyolhFFRPS29E5d1px6ymJ2RPSq9E5dNlyrTnHNiCIielVHvZOkPZK+LukpSYMlb5GkL5X8z0s6o+TPkrSp5O+QdNUo9f6mpG9KelrSR0veBZL+oXzXU5LWTUA7e9ZQvTq8L/dRRESv6uQo1KarbR9seb0BuM3245JWALcDdwG3Ati+pJyr/bCky2w3WiuTdDWwHPgx24ebZ3AXz9lefDwNmmyGyogii9kR0avG0ztdBDxR0luBG0p6IfAYgO39wCFgoM3nPwD8nu3DLWWnnKFsj42IHtdp72TgUUnbJa0seTuB60v6RmBuSe8AlkuaIWkesKTlvVYLgCskfUXS45Iua3lvnqQnS/4Vx9SiSSaL2RHR6zqderrc9r4yPbRV0m5gBfAJSXcDDwFDpexG4O3AIPBtYBtQG+G73wS8A7gMuF/ShcBLwPm2X5a0BPicpIttv9r64RKwVgKcf/75HTe41wxne2xE9LiOeifb+8rf/cBmYKnt3baX2V4C3Ac8V8rUbK+xvdj2cuBM4Jk21e4FHnTlq0ADOMv2Ydsvl7q2l3oXtLmm9bYHbA/MmTPnWNvdMw43p54yooiIHjVm7yTpdEmzm2lgGbCzufgsaRpwJ7CuvD6tlEPStUDN9q42VX8OuKaUWwDMAg5KmiNpesm/EJgPPD+uVvaw5ohiVkYUEdGjOpl6OgfYLKlZ/l7bj0haLWlVKfMgsKmkzwa2SGoALwI3NyuStAFYZ3uQaopqo6SdVNNW77NtSVcCvyupBtSB99v+7rhb2qN+8AiPBIqI6FFjBgrbzwOL2uSvBda2yd9DtSOqXV23tKSHgF9qU+YB4IGxrqtfDOdZTxHR49I7ddmRO7Nzw11E9KYEii47XG8wTTB9WgJFRPSmBIouG641mDl9GmUNKCKi5yRQdNlQvZF7KCKip6WH6rLhWiML2RHR09JDddlwRhQR0ePSQ3XZ4Vojz3mKiJ6WHqrLqjWKLGRHRO9KoOiyTD1FRK9LD9VlQ1nMjogelx6qy4brzogiInpaeqguG8pidkT0uPRQXZYb7iKi16WH6rIsZkdEr0sP1WVZzI6IXpceqsuGcx9FRPS4jgKFpD2Svi7pKUmDJW+RpC+V/M9LOqPkz5K0qeTvkHTVKPX+pqRvSnpa0kdb8u+Q9Gx575+Ps409LYvZEdHrOjkKtelq2wdbXm8AbrP9uKQVwO3AXcCtALYvKedqPyzpMtuN1sokXQ0sB37M9uGWM7gXAjcBFwNvBb4gaYHt+nG2sWfVG6ZhskYRET1tPD3URcATJb0VuKGkFwKPAdjeDxwCBtp8/gPA79k+3FIWquDxGduHbX8LeBZYOo7r7FlDtSp2JlBERC/rtIcy8Kik7ZJWlrydwPUlfSMwt6R3AMslzZA0D1jS8l6rBcAVkr4i6XFJl5X8c4EXWsrtLXmvIWmlpEFJgwcOHOiwGb1lKOdlR8Qk0OnU0+W295Xpoa2SdgMrgE9Iuht4CBgqZTcCbwcGgW8D24DaCN/9JuAdwGXA/ZIuBNqt7Pp1GfZ6YD3AwMDA696fDIbrGVFERO/rKFDY3lf+7pe0GVhq+x5gGYCkBcC7SpkasKb5WUnbgGfaVLsXeNC2ga9KagBnlfzWEch5wL5jbNek0Jx6ymJ2RPSyMXsoSadLmt1MUwWHnS2Lz9OAO4F15fVppRySrgVqtne1qfpzwDWl3AJgFnCQanRyk6RTytTVfOCr42plj8qIIiImg05GFOcAmyU1y99r+xFJqyWtKmUeBDaV9NnAljJCeBG4uVmRpA3AOtuDVFNUGyXtpJq2el8ZXTwt6X5gF9WU1ap+3PEErYvZuY8iInrXmIHC9vPAojb5a4G1bfL3UO2IalfXLS3pIeCXRij3EeAjY13bZNdczD5l5vQuX0lExMgy59FFw/VqDT4jiojoZQkUXdScepqVNYqI6GHpobpoOPdRRMQkkB6qi3JndkRMBumhuih3ZkfEZJAeqotyH0VETAbpobooi9kRMRmkh+qiLGZHxGSQHqqLhmoNpgmmT8t9FBHRuxIoumio7owmIqLnpZfqouq87PwEEdHb0kt10VCtkRFFRPS89FJdlBFFREwG6aW6aKjWyNbYiOh56aW6aKieqaeI6H0d9VKS9kj6uqSnJA2WvEWSvlTyPy/pjJI/S9Kmkr9D0lUj1Pnbkl4sdT4l6Z0l/wJJ/9CSv26C2tpzqqmnbI2NiN7W0ZnZxdW2D7a83gDcZvtxSSuA24G7gFsBbF9Sjkt9WNJlthtt6vyDcvb20Z6zvfgYrm1SymJ2REwG4+mlLgKeKOmtwA0lvRB4DMD2fuAQMDCO7+lbw3VnMTsiel6nvZSBRyVtl7Sy5O0Eri/pG4G5Jb0DWC5phqR5wJKW9472G5K+JmmjpDe15M+T9KSkxyVd0XlzJpcsZkfEZNBpL3W57UuB64BVkq4EVpT0dmA2MFTKbgT2AoPAx4FtQK1NnZ8EfgRYDLwEfKzkvwScb/vHgX8F3Ntc/2glaaWkQUmDBw4c6LAZvSWL2RExGXTUS9neV/7uBzYDS23vtr3M9hLgPuC5UqZme43txbaXA2cCz7Sp8zu262Xt4o+ApSX/sO2XS3p7qXdBm8+vtz1ge2DOnDnH3vIekPsoImIyGLOXknS6pNnNNLAM2FkWqpE0DbgTWFden1bKIelaoGZ7V5t639Ly8mepprKQNEfS9JK+EJgPPH/cLexhWcyOiMmgk11P5wCbJTXL32v7EUmrJa0qZR4ENpX02cAWSQ3gReDmZkWSNgDrbA8CH5W0mGr9Yw/w66XYlcDvSqoBdeD9tr87jjb2rIwoImIyGDNQ2H4eWNQmfy2wtk3+HqodUe3quqUlffMIZR4AHhjruvpBtZid+ygiorfln7NdUm+YhnNoUUT0vvRSXdI8BjVTTxHR69JLdclQjkGNiEkivVSXNM/LzogiInpdeqkuaU495c7siOh16aW6ZDhTTxExSaSX6pIsZkfEZJFeqkuymB0Rk0V6qS4ZrhsgBxdFRM9LoOiSLGZHxGSRXqpLspgdEZNFeqkuyWJ2REwW6aW6JIvZETFZpJfqktyZHRGTRSfnUfStPQf/jt/f8s2ufPer3x8GspgdEb1vSgeKWTOmcd6bT+3St5/KW854A2+YmUAREb2to0AhaQ/wPaoT52q2ByQtojr+9I1UJ9S91/arkmYBnwIGgAaw2vYX29T528CtwIGS9Vu2/7y8dwfwa+X7Pmh7y3G2b1RvPfNU7rju7Sei6oiIvnEsI4qrbR9seb0BuM3245JWALcDd1F1/ti+pJyr/bCky2w32tT5B7bvac2QtBC4CbgYeCvwBUkLbNeP4VojImKCjGfe4yLgiZLeCtxQ0guBxwBs7wcOUY0uOrUc+Iztw7a/BTwLLB3HdUZExDh0GigMPCppu6SVJW8ncH1J3wjMLekdwHJJMyTNA5a0vHe035D0NUkbJb2p5J0LvNBSZm/Ji4iILug0UFxu+1LgOmCVpCuBFSW9HZgNDJWyG6k690Hg48A2oNamzk8CPwIsBl4CPlby2z38yEdnSFopaVDS4IEDB9p8JCIiJkJHgcL2vvJ3P7AZWGp7t+1ltpcA9wHPlTI122tsL7a9HDgTeKZNnd+xXS9rF3/Ekemlvbx2BHIesK/N59fbHrA9MGfOnE7bGxERx2jMQCHpdEmzm2lgGbCzLFQjaRpwJ9UOKCSdVsoh6VqqXVK72tT7lpaXP0s1lQXwEHCTpFPK1NV84KvH2b6IiBinTnY9nQNsltQsf6/tRyStlrSqlHkQ2FTSZwNbJDWAF4GbmxVJ2gCssz0IfFTSYqpppT3ArwPYflrS/cAuqimrVdnxFBHRPbJfN/0/6QwMDHhwcLDblxERMalI2m57zF2pfREoJB0Avj2OKs4CDo5Zqr9MxTbD1Gx32jx1HGu732Z7zEXevggU4yVpsJOo2k+mYptharY7bZ46TlS786ChiIgYVQJFRESMKoGisr7bF9AFU7HNMDXbnTZPHSek3VmjiIiIUWVEERERo5rSgULSz0j6pqRnJX2429dzIkiaK+kvJH1D0tOSVpf8N0vaKumZ8vdNY9U1GUmaLulJSf+jvJ4n6Sul3Z8t56f0DUlnSvozSbvLb/4TU+G3lrSm/Pe9U9J9kt7Qj791eYDqfkk7W/La/r6qfKL0b1+TdOnxfu+UDRSSpgP/mepBhwuB95SzMPpNDfjXtt8OvIPqQY4LgQ8Dj9meT/VY+L4MlMBq4Bstr/8D1Tko84H/R3VAVj9ZCzxi+x8Di6ja3te/taRzgQ8CA7b/CTCd6kybfvyt/yvwM0fljfT7Xkf1CKT5wEqqB7EelykbKKgeQvis7edtDwGfoToLo6/Yfsn2X5f096g6jnOp2vrHpdgfA/+yO1d44kg6D3gX1SFbqHoOzTXAn5UifdVuSWcAVwKfBrA9ZPsQU+C3pnq80KmSZgCnUT2Ruu9+a9tPAN89Knuk33c58CeufBk486hn7HVsKgeKKXfuhaQLgB8HvgKcY/slqIIJ1TO6+s3HgX9DdSQvwA8Dh2w3H3vfb7/5hVRHC28q020bygM6+/q3tv0icA/wf6kCxCvAdvr7t2410u87YX3cVA4UHZ170S8kvRF4APiQ7Ve7fT0nmqR3A/ttb2/NblO0n37zGcClwCdt/zjwd/TZNFM7ZU5+OTCP6vjk06mmXY7WT791Jybsv/epHCg6OveiH0iaSRUk/pvtB0v2d5rD0PJ3f7eu7wS5HLhe0h6qacVrqEYYZ5bpCei/33wvsNf2V8rrP6MKHP3+W/808C3bB2wPUz3N+p/S3791q5F+3wnr46ZyoPjfwPyyM2IW1eLXQ12+pglX5uU/DXzD9n9seesh4H0l/T7gv5/sazuRbN9h+zzbF1D9tv/L9nuBvwB+vhTrq3bb/hvgBUkXlax/RvW4/r7+rammnN5RzsIRR9rdt7/1UUb6fR8CfrnsfnoH8EpziupYTekb7iS9k+pfmdOBjbY/0uVLmnCSfhL4S+DrHJmr/y2qdYr7gfOp/o92o+2jF8n6gqSrgNtsv1vShVQjjDcDTwK/ZPtwN69vIpUzXjYAs4DngV+l+gdhX//Wkn4H+EWqXX5PArdQzcf31W8t6T7gKqqnxH4H+HfA52jz+5ag+YdUu6T+HvjVchbQsX/vVA4UERExtqk89RQRER1IoIiIiFElUERExKgSKCIiYlQJFBERMaoEioiIGFUCRUREjCqBIiIiRvX/ARBAbb33BQ8XAAAAAElFTkSuQmCC

In [ ]:
vtt = h5.ts2sec(data['data']['ts'], is_packeted=True)

In [ ]:
plt.plot(vtt[1:9])

In [ ]:
plt.hist(trace,bins=70)

In [ ]:
reso.Activity.Trace()

In [ ]:
clipkey=(reso.ScanInfo & 'fps>300').fetch('KEY')[1]
clipkey


In [ ]:
newt=(reso.Activity.Trace & clipkey).fetch('trace')

In [ ]:
plt.plot(newt[0])

In [ ]:
plt.hist(newt[0])

In [ ]:
# fit to exp decay
# plot hist, decidce threshold
# thresholding, add some regulation about timing.



In [ ]:
Rastercorrection()

In [ ]:
Motioncorrection()